In [37]:
from pyaxis import pyaxis
from datetime import datetime
import pandas as pd
import predhelp as ph

def dict_clean (px_data_url, regions):
    px = rf'{px_data_url}'
    px = pyaxis.parse(uri = px , encoding = 'ISO-8859-2')
    df = px['DATA']
    dict_meta = px['METADATA']
    bad_countries = ['Übriges Südamerika',
                     'Übriges Zentralamerika, Karibik',
                     'Übriges Nordafrika',
                     'Übriges Afrika',
                     'Übriges Westasien',
                     'Übriges Süd- und Ostasien',
                     'Übriges Europa']
    for m in range(0, len(bad_countries)):
        df = df.loc[df['Herkunftsland'] != bad_countries[m]]
    df = df.loc[(df['Indikator'] == 'Logiernächte') & (df['DATA'] != '"..."') & (df['DATA'] != '"......"')]
    df["Jahr"] = df["Jahr"].astype(int)
    df["DATA"] = df["DATA"].astype(int)
    df_months = df.loc[(df['Monat'] != 'Jahrestotal')]
    df_months = df_months.groupby('Gemeinde')
    all_regions = df_months['Gemeinde'].unique()
    dict_regions = [{} for k in range(0, len(regions))]
    for i in range(0, len(regions)):
        dict_regions[i][regions[i]] = df_months.get_group(regions[i])
        dict_regions[i][regions[i]].reset_index(inplace = True)
        dict_regions[i][regions[i]].rename(columns = {'Jahr' : 'year', 'Monat': 'month'}, inplace = True)
    trans_dict = {
        'Schweiz' : 'Switzerland',
        'Deutschland' : 'Germany',
        'Italien': 'Italy',
        'Frankreich' : 'France',
        'Österreich' : 'Austria',
        'Vereinigtes Königreich': 'United Kingdom',
        'Irland' : 'Ireland',
        'Niederlande' : 'Netherlands',
        'Belgien' : 'Belgium',
        'Luxemburg' : 'Luxembourg',
        'Dänemark' : 'Denmark',
        'Schweden' : 'Sweden',
        'Norwegen' : 'Norway',
        'Finnland' : 'Finland',
        'Spanien' : 'Spain',
        'Griechenland' : 'Greece',
        'Türkei' : 'Turkey',
        'Island' : 'Iceland',
        'Polen' : 'Polony',
        'Ungarn' : 'Ungary',
        'Bulgarien': 'Bulgary',
        'Zypern' : 'Cyprus',
        'Vereinigte Staaten' : 'United States',
        'Kanada' : 'Canada',
        'Mexiko' : 'Mexico',
        'Brasilien' : 'Brasil',
        'Portugal' : 'Portugal',
        'Argentinien' : 'Argentine',
        'Ägypten' : 'Egypt',
        'Südafrika' : 'South Africa',
        'Indien' : 'India',
        'Katar' : 'Qatar',
        'Asutralien' : 'Australia',
        'Indonesien' : 'Indonesia',
        'Korea (Süd-)' : 'South Korea',
        'Phillipinen' : 'Philippines',
        'Neuseeland, Ozeanien' : 'New Zealand',
        'Singapur' : 'Singapore',
        'Taiwan (Chinesisches Taipei)' : 'Taiwan',
        'Estland' : 'Estonia',
        'Lettland' : 'Latvia',
        'Litauen' : 'Lithuania',
        'Saudi-Arabien' : 'Saudi Arabia',
        'Vereinigte Arabische Emirate' : 'United Arab Emirates',
        'Kroatien' : 'Croatia',
        'Rumänien' : 'Romania',
        'Russland' : 'Russia',
        'Slowakei' : 'Slovakia',
        'Slowenien' : 'Slovenia',
        'Tschechien' : 'Czech Republic',
        'Serbien' : 'Serbia',
        }
    
    for k in range(0, len(dict_regions)):
        for i in range(0, len(dict_regions[k][regions[k]])):
            if dict_regions[k][regions[k]].iloc[i]['Herkunftsland'] in trans_dict.keys():
                dict_regions[k][regions[k]].at[i,'Herkunftsland'] = trans_dict[dict_regions[k][regions[k]].iloc[i]['Herkunftsland']]
    trans_months = {
        'Januar' : 1,
        'Februar' : 2,
        'März' : 3,
        'April' : 4,
        'Mai' : 5,
        'Juni' : 6,
        'Juli' : 7,
        'August' : 8,
        'September' : 9,
        'Oktober' : 10,
        'November' : 11,
        'Dezember' : 12}
    for k in range(0, len(dict_regions)):
        for i in range(0, len(dict_regions[k][regions[k]])):
            if dict_regions[k][regions[k]].iloc[i]['month'] in trans_months.keys():
                dict_regions[k][regions[k]].at[i,'month'] = trans_months[dict_regions[k][regions[k]].iloc[i]['month']]
    for i in range(0, len(dict_regions)):
        dict_regions[i][regions[i]]['date'] = pd.to_datetime(dict_regions[i][regions[i]][['year','month']].assign(DAY = 1))
        dict_regions[i][regions[i]].drop(labels = ['year', 'month'], axis = 1, inplace = True)
    for i in range(0, len(dict_regions)):
        dict_regions[i][regions[i]] = dict_regions[i][regions[i]].pivot(index =  ['date'],columns = 'Herkunftsland', values = 'DATA').reset_index().rename_axis(None, axis=1)
     
    return(dict_regions, all_regions)

In [32]:

#gdp = pd.read_csv("../data/cleaned data/gdp.csv")
#exchange_rate = pd.read_csv("../data/cleaned data/exchangerate.csv")
#PPI = pd.read_csv("../data/cleaned data/PPI.csv")
#unemployment = pd.read_csv("../data/cleaned data/unemployment.csv")
regions = ["Saanen"]
df_final, all_regions = dict_clean("../data/px-x-1003020000_101.px", regions)


/var/folders/bn/v98ltrs170dcrqbn6tc0qdb00000gn/T/ipykernel_25192/1533828624.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_regions[i][regions[i]].rename(columns = {'Jahr' : 'year', 'Monat': 'month'}, inplace = True)
/var/folders/bn/v98ltrs170dcrqbn6tc0qdb00000gn/T/ipykernel_25192/1533828624.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_regions[i][regions[i]]['date'] = pd.to_datetime(dict_regions[i][regions[i]][['year','month']].assign(DAY = 1))
/var/folders/bn/v98ltrs170dcrqbn6tc0qdb00000gn/T/ipykernel_25192/1533828624.py:108: SettingWithC

In [34]:
df_final[0]["Saanen"]

,date,Argentine,Australien,Austria,Bahrain,Belarus,Belgium,Brasil,Bulgary,Canada,...,Sweden,Switzerland,Taiwan,Thailand,Turkey,Ukraine,Ungary,United Arab Emirates,United Kingdom,United States
0,2013-01-01,8.0,218.0,98.0,10.0,32.0,721.0,331.0,9.0,159.0,...,52.0,21329.0,0.0,759.0,269.0,237.0,26.0,179.0,1610.0,1059.0
1,2013-02-01,12.0,13.0,180.0,21.0,2.0,2023.0,323.0,176.0,82.0,...,260.0,24952.0,37.0,132.0,299.0,62.0,12.0,153.0,3210.0,1317.0
2,2013-03-01,14.0,18.0,37.0,10.0,0.0,819.0,77.0,8.0,58.0,...,215.0,18544.0,0.0,25.0,46.0,119.0,4.0,100.0,1150.0,792.0
3,2013-04-01,4.0,34.0,5.0,14.0,0.0,545.0,16.0,0.0,2.0,...,4.0,4619.0,8.0,65.0,0.0,2.0,0.0,23.0,87.0,105.0
4,2013-05-01,0.0,17.0,20.0,0.0,375.0,233.0,6.0,0.0,24.0,...,0.0,6637.0,0.0,30.0,3.0,6.0,1.0,1.0,287.0,131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2022-05-01,0.0,10.0,32.0,0.0,0.0,60.0,13.0,3.0,36.0,...,4.0,12698.0,0.0,11.0,8.0,32.0,2.0,39.0,225.0,276.0
113,2022-06-01,11.0,36.0,192.0,9.0,2.0,175.0,13.0,0.0,97.0,...,22.0,20482.0,0.0,43.0,13.0,32.0,10.0,136.0,607.0,1206.0
114,2022-07-01,31.0,271.0,156.0,62.0,2.0,555.0,240.0,18.0,276.0,...,86.0,30553.0,25.0,189.0,89.0,39.0,72.0,1154.0,1427.0,2732.0
115,2022-08-01,68.0,131.0,123.0,32.0,8.0,365.0,106.0,9.0,121.0,...,38.0,27512.0,12.0,37.0,127.0,35.0,29.0,1378.0,2320.0,2156.0


In [ ]:
#do a logistic regression

#first check if we need we don't have any more objects left
df.info()
#statsmodels requires all variables to be floats
df = df.astype(float)


import statsmodels.api as sm #statsmodels is used for creating linear regression models
from sklearn.model_selection import train_test_split #creating testing and training data
from sklearn.metrics import roc_auc_score #evaluate model performance

#define the y and x
y = df["Logiernächte"]
X = loan.drop("Logiernächte", axis = 1)

#creating test and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2




